# gh

> Command line utilities for github

In [ ]:
#| default_exp gh

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import itertools as it
import os
from ghapi.all import *

To use this module you need to setup GitHub [personal access token](https://docs.github.com/github/authenticating-to-github/creating-a-personal-access-token), which is a secret code used to access your account. If you don't have one, [click here](https://github.com/settings/tokens/new) to create one. You'll be asked to enter a name -- choose anything you like, for instance "oztools".
In "scopes" choose "repo", "gist", "notifications", and "workflow". Then clock "Generate Token" and copy the token. Then add it for example to your `.bashrc` as `export GITHUB_TOKEN="<your github token goes here>"`.

In [ ]:
import configparser
from pathlib import Path

In [ ]:
configpath = Path("~/.gitconfig").expanduser()

config = configparser.ConfigParser()
config.read(configpath)

gh_username = config['user']['name']

In [ ]:
token = os.environ["GITHUB_TOKEN"]

In [ ]:
api = GhApi(owner=gh_username, token=token)

In [ ]:
api.git.get_ref(repo='oztools', ref='heads/main')

```json
{ 'node_id': 'REF_kwDONy_jba9yZWZzL2hlYWRzL21haW4',
  'object': { 'sha': '8a23e2af6915688ab9947858e904077ef62bbda0',
              'type': 'commit',
              'url': 'https://api.github.com/repos/ozellpaukert/oztools/git/commits/8a23e2af6915688ab9947858e904077ef62bbda0'},
  'ref': 'refs/heads/main',
  'url': 'https://api.github.com/repos/ozellpaukert/oztools/git/refs/heads/main'}
```

In [ ]:
api.git

- [git.create_blob](https://docs.github.com/rest/git/blobs#create-a-blob)(repo, content, encoding): *Create a blob*
- [git.get_blob](https://docs.github.com/rest/git/blobs#get-a-blob)(repo, file_sha): *Get a blob*
- [git.create_commit](https://docs.github.com/rest/git/commits#create-a-commit)(repo, message, tree, parents, author, committer, signature): *Create a commit*
- [git.get_commit](https://docs.github.com/rest/git/commits#get-a-commit-object)(repo, commit_sha): *Get a commit object*
- [git.list_matching_refs](https://docs.github.com/rest/git/refs#list-matching-references)(repo, ref): *List matching references*
- [git.get_ref](https://docs.github.com/rest/git/refs#get-a-reference)(repo, ref): *Get a reference*
- [git.create_ref](https://docs.github.com/rest/git/refs#create-a-reference)(repo, ref, sha): *Create a reference*
- [git.update_ref](https://docs.github.com/rest/git/refs#update-a-reference)(repo, ref, sha, force): *Update a reference*
- [git.delete_ref](https://docs.github.com/rest/git/refs#delete-a-reference)(repo, ref): *Delete a reference*
- [git.create_tag](https://docs.github.com/rest/git/tags#create-a-tag-object)(repo, tag, message, object, type, tagger): *Create a tag object*
- [git.get_tag](https://docs.github.com/rest/git/tags#get-a-tag)(repo, tag_sha): *Get a tag*
- [git.create_tree](https://docs.github.com/rest/git/trees#create-a-tree)(repo, tree, base_tree): *Create a tree*
- [git.get_tree](https://docs.github.com/rest/git/trees#get-a-tree)(repo, tree_sha, recursive): *Get a tree*

In [ ]:
#api.repos.list_for_org("")